# 🧠 Sentiment Analysis with Fine-Tuned DistilBERT
### Complete End-to-End Project on Google Colab

**What we should do is:**
1. Install dependencies & verify GPU
2. Load & explore the IMDb dataset
3. Tokenize text for DistilBERT
4. Load pretrained model & add classifier head
5. Train with HuggingFace Trainer API
6. Evaluate with metrics + confusion matrix
7. Save model to Google Drive
8. Run predictions + interactive widget

> ⏱ Total runtime: ~10–15 minutes on T4 GPU


## Step 1: Install Dependencies

In [ ]:
!pip install transformers datasets evaluate accelerate scikit-learn -q
print('✅ All packages installed!')

## Step 2: Imports & GPU Check

We import all necessary libraries and verify that GPU is available.  
`fp16=True` (mixed precision) only works on GPU — this speeds up training ~2x.


In [ ]:
import torch
import numpy as np
import evaluate
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import load_dataset, DatasetDict
from sklearn.metrics import classification_report, confusion_matrix

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'GPU available: {torch.cuda.is_available()}')
print(f'Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU — go to Runtime > Change runtime type > T4 GPU"}')
print(f'PyTorch version: {torch.__version__}')

## Step 3: Load & Explore Dataset

We use the **IMDb movie review dataset** — 50,000 reviews labeled POSITIVE/NEGATIVE.

We take a **subset** (3000 train / 750 test) for speed on the free Colab tier.  
Remove `.select()` to train on the full dataset (~25 min, better accuracy).


In [ ]:
MODEL_NAME = 'distilbert-base-uncased'

print('Loading IMDb dataset...')
raw = load_dataset('imdb')

# Subset for speed — remove .select() for full training
train_data = raw['train'].shuffle(seed=42).select(range(3000))
test_data  = raw['test'].shuffle(seed=42).select(range(750))

dataset = DatasetDict({'train': train_data, 'test': test_data})

print(f'Train: {len(dataset["train"])} samples')
print(f'Test:  {len(dataset["test"])} samples')
print(f'Labels: {set(dataset["train"]["label"])}  (0=NEGATIVE, 1=POSITIVE)')

# Check label balance
labels = dataset['train']['label']
pos = sum(labels)
neg = len(labels) - pos
print(f'\nClass balance — POSITIVE: {pos} ({pos/len(labels):.0%}) | NEGATIVE: {neg} ({neg/len(labels):.0%})')

# Show a sample
print(f'\nSample review (truncated):')
print(dataset['train'][0]['text'][:300], '...')

## Step 4: Tokenize

Tokenization converts raw text → numbers the model understands.

- **`input_ids`**: token IDs (e.g. "great" → 2307)
- **`attention_mask`**: 1 for real tokens, 0 for padding
- **`max_length=256`**: truncates/pads all sequences to same length

DistilBERT uses **WordPiece tokenization** — unknown words are split into subwords.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Show what tokenization looks like
example = 'This movie was absolutely fantastic!'
tokens = tokenizer.tokenize(example)
ids    = tokenizer.encode(example)
print(f'Text:   {example}')
print(f'Tokens: {tokens}')
print(f'IDs:    {ids}')
print()

# Tokenize full dataset
def tokenize_fn(batch):
    return tokenizer(
        batch['text'],
        padding='max_length',
        truncation=True,
        max_length=256
    )

print('Tokenizing dataset...')
tokenized = dataset.map(tokenize_fn, batched=True)
tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
print('Done! Keys:', list(tokenized['train'][0].keys()))

## Step 5: Load Pretrained Model

`distilbert-base-uncased` is a **smaller, faster version of BERT**:
- 40% fewer parameters than BERT (66M vs 110M)
- 60% faster inference
- Retains 97% of BERT's performance

We added a **classification head** on top — a fresh linear layer mapping from 768 dimensions → 2 labels (POSITIVE / NEGATIVE).

The `MISSING` keys in the output are expected — that's the new classification head being randomly initialized (it will be trained).


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    id2label={0: 'NEGATIVE', 1: 'POSITIVE'},
    label2id={'NEGATIVE': 0, 'POSITIVE': 1},
    ignore_mismatched_sizes=True  # suppresses the size mismatch warning
)

total     = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total parameters:     {total:,}')
print(f'Trainable parameters: {trainable:,}')
print(f'Model architecture:\n{model.__class__.__name__}')

## Step 6: Define Metrics

We track two metrics:
- **Accuracy**: % of correct predictions
- **F1 Score**: harmonic mean of precision & recall — better for imbalanced datasets

We use F1 as the metric to pick the best checkpoint.


In [ ]:
accuracy_metric = evaluate.load('accuracy')
f1_metric       = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        **accuracy_metric.compute(predictions=preds, references=labels),
        **f1_metric.compute(predictions=preds, references=labels)
    }

print('✅ Metrics defined: accuracy + F1')

## Step 7: Configure Training Arguments

Key hyperparameter choices:

| Parameter | Value | Why |
|---|---|---|
| `learning_rate` | 2e-5 | Small — we nudge pretrained weights, not retrain |
| `epochs` | 3 | Usually 2–4 is enough for fine-tuning |
| `warmup_steps` | 56 | Gradually ramps up LR at start to avoid instability |
| `weight_decay` | 0.01 | L2 regularization to reduce overfitting |
| `fp16` | auto | Mixed precision on GPU = faster + less memory |
| `load_best_model_at_end` | True | Restores best checkpoint after training |


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=56,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='steps',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    fp16=torch.cuda.is_available(),
    seed=42,
    report_to='none'
)
print('✅ Training arguments configured')

## Step 8: Train! 🚀

This is the core step. The Trainer handles:
- Forward pass → compute loss
- Backward pass → compute gradients (backprop)
- Optimizer step → update weights
- Evaluation after each epoch

**Expected time: ~8–12 minutes on T4 GPU**


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

print('Starting training...')
train_result = trainer.train()

print(f'\n✅ Training complete!')
print(f'Total train time: {train_result.metrics["train_runtime"]:.1f}s')
print(f'Samples/second:   {train_result.metrics["train_samples_per_second"]:.1f}')

## Step 9: Evaluate Model

We run a full evaluation including:
- Final metrics (accuracy, F1, loss)
- Classification report (precision, recall per class)
- Confusion matrix visualization


In [ ]:
# Final metrics
results = trainer.evaluate()
print('Final Evaluation Results:')
print(f'  Accuracy: {results["eval_accuracy"]:.4f}')
print(f'  F1 Score: {results["eval_f1"]:.4f}')
print(f'  Loss:     {results["eval_loss"]:.4f}')

# Full inference loop for classification report
model.eval()
all_preds, all_labels = [], []

for i in range(0, len(tokenized['test']), 32):
    batch = tokenized['test'][i:i+32]
    with torch.no_grad():
        outputs = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device)
        )
    all_preds.extend(torch.argmax(outputs.logits, dim=-1).cpu().tolist())
    all_labels.extend(batch['label'].tolist())

print('\nClassification Report:')
print(classification_report(all_labels, all_preds, target_names=['NEGATIVE', 'POSITIVE']))

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['NEGATIVE', 'POSITIVE'],
            yticklabels=['NEGATIVE', 'POSITIVE'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.tight_layout()
plt.show()

# Training loss curve
log_history = trainer.state.log_history
train_losses = [(x['step'], x['loss']) for x in log_history if 'loss' in x]
if train_losses:
    steps, losses = zip(*train_losses)
    plt.figure(figsize=(8, 4))
    plt.plot(steps, losses, marker='o', markersize=3, linewidth=1.5, color='royalblue')
    plt.title('Training Loss Curve')
    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## Step 10: Save Model to Google Drive

**Important!** Colab resets after ~90 min of inactivity — the model will be lost.  
We should always save to Google Drive so you can reload it in a new session.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MODEL_SAVE_PATH = '/content/drive/MyDrive/sentiment_model'
trainer.save_model(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)
print(f'✅ Model saved to Google Drive: {MODEL_SAVE_PATH}')

## Step 11: Load & Run Predictions

We used the `pipeline` API — the simplest way to use any HuggingFace model.  



In [ ]:
from transformers import pipeline

MODEL_SAVE_PATH = '/content/drive/MyDrive/sentiment_model'

classifier = pipeline(
    'text-classification',
    model=MODEL_SAVE_PATH,
    return_all_scores=True,
    device=0 if torch.cuda.is_available() else -1
)

test_sentences = [
    'This movie was absolutely fantastic, I loved every minute!',
    'Terrible film. Boring, predictable and poorly acted.',
    'It was okay, not great but not terrible either.',
    'One of the best performances I\'ve seen in years.',
    'Complete waste of time and money.',
    'A beautiful story with stunning visuals.',
    'I fell asleep halfway through, so dull.'
]

print('Predictions:\n')
for text in test_sentences:
    all_scores = classifier(text)
    scores = {r['label']: r['score'] for r in all_scores}
    label  = max(scores, key=scores.get)
    conf   = scores[label]
    emoji  = '😊' if label == 'POSITIVE' else '😞'
    bar_pos = '█' * int(scores['POSITIVE'] * 20)
    bar_neg = '█' * int(scores['NEGATIVE'] * 20)
    print(f'{emoji} [{label} — {conf:.1%}]  {text[:60]}')
    print(f'   POS [{bar_pos:<20}] {scores["POSITIVE"]:.1%}')
    print(f'   NEG [{bar_neg:<20}] {scores["NEGATIVE"]:.1%}\n')

## Step 12: Interactive Demo Widget

Typed any text and click **Analyze** to see the model's prediction live!


In [ ]:
from ipywidgets import widgets
from IPython.display import display, clear_output

text_input = widgets.Textarea(
    placeholder='Type any sentence or movie review here...',
    layout=widgets.Layout(width='600px', height='100px')
)
button = widgets.Button(
    description='Analyze Sentiment',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)
output = widgets.Output()

def on_click(b):
    with output:
        clear_output()
        text = text_input.value.strip()
        if not text:
            print('⚠️ Please enter some text!')
            return
        all_scores = classifier(text)
        scores = {r['label']: r['score'] for r in all_scores}
        label  = max(scores, key=scores.get)
        conf   = scores[label]
        emoji  = '😊' if label == 'POSITIVE' else '😞'
        print(f'\nResult: {emoji} {label}')
        print(f'Confidence: {conf:.1%}')
        print(f'  POSITIVE: {scores.get("POSITIVE", 0.0):.1%}')
        print(f'  NEGATIVE: {scores.get("NEGATIVE", 0.0):.1%}')

button.on_click(on_click)
display(text_input, button, output)